# Cleaning NHS sitreps data

NHS England publish 'sitreps' on discharge delays at https://www.england.nhs.uk/statistics/statistical-work-areas/discharge-delays-acute-data/ - we have already fetched and cleaned data in a separate notebook. In this notebook we need to do the same for a different table in the same spreadsheets:

* Fetch the data files
* Reshape the data
* Combine it
* Question it

Let's start by importing the libraries we will need.

## Import the libraries

We need the `pandas` library to import a CSV, convert to a dataframe and reshape it, and the `requests` and `bs4` libraries for scraping the CSVs.

In [ ]:
#import the pandas library and rename as pd
import pandas as pd
#requests for fetching webpages
import requests
#BeautifulSoup for scraping
from bs4 import BeautifulSoup
#regex for cleaning
import re

## Import the data

The data is published in two ways:

* An XLSX file with 3 tables, and 5 sheets
* A CSV file with the second table from that XLSX file in the first 127 columns, then the third table 131 columns in (after 4 empty columns)

The column headings in both don't appear until row 4 (and row 5 for table 3 in the CSV), so we skip the first 3 rows in the import.

For the CSV we also limit it to the first 127 columns. To do this we use the `usecols=` parameter, and generate a `range` of indices to give it. We have to add `*` to ["'unpick' the iterator"](https://stackoverflow.com/questions/35400861/pandas-read-table-usecols-error-with) returned by `range`.

Here's the code for the CSV - but it isn't going to be much use...

In [ ]:
#store the URL of one CSV file
csvurl = "https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2023/04/Daily-discharge-sitrep-monthly-data-webfile-March2023.csv"
#read it in, skipping the first 3 rows
#see https://stackoverflow.com/questions/35400861/pandas-read-table-usecols-error-with on how to specify a range of cols
df = pd.read_csv(csvurl, skiprows = 3,
                 skipinitialspace=True,
                 usecols=[*range(0,127)]) #import 127 cols only
#and show the first 5 rows
df.head(5)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,01/03/2023,Unnamed: 4,Unnamed: 5,Unnamed: 6,02/03/2023,Unnamed: 8,Unnamed: 9,...,Unnamed: 117,Unnamed: 118,30/03/2023,Unnamed: 120,Unnamed: 121,Unnamed: 122,31/03/2023,Unnamed: 124,Unnamed: 125,Unnamed: 126
0,NaN,NaN,NaN,Number of patients who no longer meet the crit...,Number of patients discharged by 17:00,Number of patients discharged between 17:01 an...,Number of patients remaining in hospital who n...,Number of patients who no longer meet the crit...,Number of patients discharged by 17:00,Number of patients discharged between 17:01 an...,...,Number of patients discharged between 17:01 an...,Number of patients remaining in hospital who n...,Number of patients who no longer meet the crit...,Number of patients discharged by 17:00,Number of patients discharged between 17:01 an...,Number of patients remaining in hospital who n...,Number of patients who no longer meet the crit...,Number of patients discharged by 17:00,Number of patients discharged between 17:01 an...,Number of patients remaining in hospital who n...
1,NaN,NaN,ENGLAND (All Acute Trusts),24374,6702,4443,13229,24187,6565,4452,...,4519,13306,24666,7006,4517,13143,25069,7134,4728,13207
2,NaN,NaN,EAST OF ENGLAND,2478,991,453,1034,2431,980,472,...,458,1032,2340,934,448,958,2352,891,469,992
3,NaN,NaN,LONDON,2887,864,608,1415,2917,858,721,...,612,1449,3119,960,683,1476,3139,955,733,1451
4,NaN,NaN,MIDLANDS,4332,1249,826,2257,4411,1231,749,...,926,2309,4378,1257,832,2289,4367,1292,838,2237


### Import the XLSX

...Because for some reason only the last few months are provided as CSV files. For all 12 months we need to import the XLSX files.

In [ ]:
#store the URL of one XLSX file
xlsxurl = "https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2022/08/Daily-discharge-sitrep-monthly-data-webfile-April2022-v2.xlsx"
#replace with the revised version that replaced it in August 2023
xlsxurl = "https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2023/08/Daily-discharge-sitrep-monthly-data-webfile-April2022-revised.xlsx"
#read in the sheet called 'Table 5', skipping the first 3 rows
#see https://stackoverflow.com/questions/35400861/pandas-read-table-usecols-error-with on how to specify a range of cols
df = pd.read_excel(xlsxurl, skiprows = 3, sheet_name = 'Table 5')
#and show the first 5 rows
df.head(5)

,Unnamed: 0,"Notes: \n1. Submitting providers choose on which day of the week their snapshot will be collected depending on local operational practices, they do not have to select the same day of the week every collection window and different submitting providers may have selected different days in any given collection window. \n2. Due to disclosure risks, trust-level and ICB-level data exclude patients who have a delay reason of ""Homeless/no right of recourse to public funds/no place to discharge to"" and ""Safeguarding concern preventing discharge or Court of protection"". \nThis may explain differing sums to regional and national totals.\n3. Due to missing provider submissions, aggregated provider figures may not total ICB, Regional or National level figures.\n4. There 4 weekly snapshots in the publication this month. Please see Table 3 for reference where there a dashes against each day for which an organisation did not provide data.",Unnamed: 2,Unnamed: 3,April 2022 - Average number of people per day with length of stay 14 days or over who no longer meet the criteria to reside but were not discharged,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,NaN,NaN,NaN,NaN,Awaiting a medical decision/intervention inclu...,Awaiting community equipment and adaptations t...,Awaiting confirmation from community hub/singl...,Awaiting Diagnostic test,Awaiting medicines to take home,Awaiting referral to community single point of...,...,Declared as not meeting the criteria to reside...,Homeless/no right of recourse to public funds/...,Individual/family not in agreement with discha...,No Plan,Pathway 1: awaiting availability of resource f...,Pathway 2: awaiting availability of rehabilita...,Pathway 3: awaiting availability of a bed in a...,Remains in hospital to avoid spread of (non-Co...,Repatriation/Transfer to another acute trust f...,Safeguarding concern preventing discharge or C...
1,NaN,NaN,NaN,ENGLAND (Type 1 Trusts),411.75,157,421.75,135.5,66.75,296.75,...,81.25,65.75,94.5,385.75,2040.75,2040,1517.25,66.25,140.25,53.5
2,NaN,NaN,NaN,EAST OF ENGLAND,42.25,30,55,40,25.25,27.25,...,1,5.75,8.5,50.75,278.5,265.25,108,8.5,27.5,2.5
3,NaN,NaN,NaN,LONDON,61,35.75,17.75,3.25,6.75,17,...,7.75,22,22,34,163.25,143.25,237.75,0.25,33,4.75
4,NaN,NaN,NaN,MIDLANDS,103.25,17.5,56.5,49,14.25,47,...,9,3.5,11.5,62.5,295.5,281.25,167.5,9.5,18.75,7


## Fetch the date and measure

The column headers are actually in row 1 - but we need to fetch the label that says what month and measure this data refers  firstto. That's now in the header row, item 4.

In [ ]:
#show the 5th column name
print(df.columns[4])
#store it
month_and_measure = df.columns[4]
print(month_and_measure)
#split it and strip out white space
monthyr = month_and_measure.split('-')[0].strip()
print(monthyr)
measure = month_and_measure.split('-')[1].strip()
print(measure)

April 2022 - Average number of people per day with length of stay 14 days or over who no longer meet the criteria to reside but were not discharged
April 2022 - Average number of people per day with length of stay 14 days or over who no longer meet the criteria to reside but were not discharged
April 2022
Average number of people per day with length of stay 14 days or over who no longer meet the criteria to reside but were not discharged


## Re-import the data

With that stored, we can re-import - skipping that row and instead using the next row and column headings.

In [ ]:
#store the URL of one XLSX file
xlsxurl = "https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2022/08/Daily-discharge-sitrep-monthly-data-webfile-April2022-v2.xlsx"
#replace with the revised version that replaced it in August 2023
xlsxurl = "https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2023/08/Daily-discharge-sitrep-monthly-data-webfile-April2022-revised.xlsx"
#read in the sheet called 'Table 5', skipping the first 4 rows
#see https://stackoverflow.com/questions/35400861/pandas-read-table-usecols-error-with on how to specify a range of cols
df = pd.read_excel(xlsxurl, skiprows = 4, sheet_name = 'Table 5')
#and show the first 5 rows
df.head(5)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Awaiting a medical decision/intervention including writing the discharge summary,Awaiting community equipment and adaptations to housing,Awaiting confirmation from community hub/single point of access that referral received and actioned,Awaiting Diagnostic test,Awaiting medicines to take home,Awaiting referral to community single point of access,...,Declared as not meeting the criteria to reside at morning board round and then later in day meets the criteria to reside so discharge stopped,Homeless/no right of recourse to public funds/no place to discharge to,Individual/family not in agreement with discharge plans,No Plan,Pathway 1: awaiting availability of resource for assessment and start of care at home,Pathway 2: awaiting availability of rehabilitation bed in community hospital or other bedded setting,Pathway 3: awaiting availability of a bed in a residential or nursing home that is likely to be a permanent placement,Remains in hospital to avoid spread of (non-Covid 19) infectious disease and because there is no other suitable location to discharge to,Repatriation/Transfer to another acute trust for specialist treatment or ongoing treatment,Safeguarding concern preventing discharge or Court of protection
0,NaN,NaN,NaN,ENGLAND (Type 1 Trusts),411.75,157.00,421.75,135.50,66.75,296.75,...,81.25,65.75,94.50,385.75,2040.75,2040.00,1517.25,66.25,140.25,53.5
1,NaN,NaN,NaN,EAST OF ENGLAND,42.25,30.00,55.00,40.00,25.25,27.25,...,1.00,5.75,8.50,50.75,278.50,265.25,108.00,8.50,27.50,2.5
2,NaN,NaN,NaN,LONDON,61.00,35.75,17.75,3.25,6.75,17.00,...,7.75,22,22.00,34.00,163.25,143.25,237.75,0.25,33.00,4.75
3,NaN,NaN,NaN,MIDLANDS,103.25,17.50,56.50,49.00,14.25,47.00,...,9.00,3.5,11.50,62.50,295.50,281.25,167.50,9.50,18.75,7
4,NaN,NaN,NaN,NORTH EAST AND YORKSHIRE,60.50,29.25,79.50,24.75,5.00,41.00,...,2.50,9.25,21.25,66.25,374.25,335.75,289.75,14.00,18.75,15


## Remove columns

We can remove the empty column.

In [ ]:
#axis = 1 means we want to drop columns
#drop column 0
df = df.drop(axis = 1, labels = df.keys()[0])
df.head(10)

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Awaiting a medical decision/intervention including writing the discharge summary,Awaiting community equipment and adaptations to housing,Awaiting confirmation from community hub/single point of access that referral received and actioned,Awaiting Diagnostic test,Awaiting medicines to take home,Awaiting referral to community single point of access,Awaiting therapy decision to discharge,...,Declared as not meeting the criteria to reside at morning board round and then later in day meets the criteria to reside so discharge stopped,Homeless/no right of recourse to public funds/no place to discharge to,Individual/family not in agreement with discharge plans,No Plan,Pathway 1: awaiting availability of resource for assessment and start of care at home,Pathway 2: awaiting availability of rehabilitation bed in community hospital or other bedded setting,Pathway 3: awaiting availability of a bed in a residential or nursing home that is likely to be a permanent placement,Remains in hospital to avoid spread of (non-Covid 19) infectious disease and because there is no other suitable location to discharge to,Repatriation/Transfer to another acute trust for specialist treatment or ongoing treatment,Safeguarding concern preventing discharge or Court of protection
0,NaN,NaN,ENGLAND (Type 1 Trusts),411.75,157.00,421.75,135.50,66.75,296.75,439.00,...,81.25,65.75,94.50,385.75,2040.75,2040.00,1517.25,66.25,140.25,53.5
1,NaN,NaN,EAST OF ENGLAND,42.25,30.00,55.00,40.00,25.25,27.25,60.00,...,1.00,5.75,8.50,50.75,278.50,265.25,108.00,8.50,27.50,2.5
2,NaN,NaN,LONDON,61.00,35.75,17.75,3.25,6.75,17.00,49.50,...,7.75,22,22.00,34.00,163.25,143.25,237.75,0.25,33.00,4.75
3,NaN,NaN,MIDLANDS,103.25,17.50,56.50,49.00,14.25,47.00,46.50,...,9.00,3.5,11.50,62.50,295.50,281.25,167.50,9.50,18.75,7
4,NaN,NaN,NORTH EAST AND YORKSHIRE,60.50,29.25,79.50,24.75,5.00,41.00,66.00,...,2.50,9.25,21.25,66.25,374.25,335.75,289.75,14.00,18.75,15
5,NaN,NaN,NORTH WEST,99.25,17.75,84.00,16.75,11.50,92.00,91.25,...,14.25,7,9.00,103.00,289.25,244.50,256.00,14.50,9.75,5.25
6,NaN,NaN,SOUTH EAST,32.25,14.00,27.25,1.50,2.75,18.75,44.00,...,16.50,11.25,11.25,62.00,293.50,343.50,235.75,6.25,11.00,3
7,NaN,NaN,SOUTH WEST,13.25,12.75,101.75,0.25,1.25,53.75,81.75,...,30.25,7,11.00,7.25,346.50,426.50,222.50,13.25,21.50,16
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Region,ICB Code,ICB Name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Clean up the headings

The column headings are in three different places here:

* The current headings show the categories
* Row 10 (15 in the original) contains the headings for the first 3 columns

We also have an empty row in row 9.


In [ ]:
#show the values in row 10
for i in df.iloc[9]:
  print(i)

Region
ICB Code
ICB Name
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [ ]:
#fetch values in first 3 cells of row 10
#and store in 3 corresponding positions in the columns
df = df.rename(columns={df.columns[0]: df.iloc[9][0],
                   df.columns[1]: df.iloc[9][1],
                   df.columns[2]: df.iloc[9][2]})
df.head(10)

,Region,ICB Code,ICB Name,Awaiting a medical decision/intervention including writing the discharge summary,Awaiting community equipment and adaptations to housing,Awaiting confirmation from community hub/single point of access that referral received and actioned,Awaiting Diagnostic test,Awaiting medicines to take home,Awaiting referral to community single point of access,Awaiting therapy decision to discharge,...,Declared as not meeting the criteria to reside at morning board round and then later in day meets the criteria to reside so discharge stopped,Homeless/no right of recourse to public funds/no place to discharge to,Individual/family not in agreement with discharge plans,No Plan,Pathway 1: awaiting availability of resource for assessment and start of care at home,Pathway 2: awaiting availability of rehabilitation bed in community hospital or other bedded setting,Pathway 3: awaiting availability of a bed in a residential or nursing home that is likely to be a permanent placement,Remains in hospital to avoid spread of (non-Covid 19) infectious disease and because there is no other suitable location to discharge to,Repatriation/Transfer to another acute trust for specialist treatment or ongoing treatment,Safeguarding concern preventing discharge or Court of protection
0,NaN,NaN,ENGLAND (Type 1 Trusts),411.75,157.00,421.75,135.50,66.75,296.75,439.00,...,81.25,65.75,94.50,385.75,2040.75,2040.00,1517.25,66.25,140.25,53.5
1,NaN,NaN,EAST OF ENGLAND,42.25,30.00,55.00,40.00,25.25,27.25,60.00,...,1.00,5.75,8.50,50.75,278.50,265.25,108.00,8.50,27.50,2.5
2,NaN,NaN,LONDON,61.00,35.75,17.75,3.25,6.75,17.00,49.50,...,7.75,22,22.00,34.00,163.25,143.25,237.75,0.25,33.00,4.75
3,NaN,NaN,MIDLANDS,103.25,17.50,56.50,49.00,14.25,47.00,46.50,...,9.00,3.5,11.50,62.50,295.50,281.25,167.50,9.50,18.75,7
4,NaN,NaN,NORTH EAST AND YORKSHIRE,60.50,29.25,79.50,24.75,5.00,41.00,66.00,...,2.50,9.25,21.25,66.25,374.25,335.75,289.75,14.00,18.75,15
5,NaN,NaN,NORTH WEST,99.25,17.75,84.00,16.75,11.50,92.00,91.25,...,14.25,7,9.00,103.00,289.25,244.50,256.00,14.50,9.75,5.25
6,NaN,NaN,SOUTH EAST,32.25,14.00,27.25,1.50,2.75,18.75,44.00,...,16.50,11.25,11.25,62.00,293.50,343.50,235.75,6.25,11.00,3
7,NaN,NaN,SOUTH WEST,13.25,12.75,101.75,0.25,1.25,53.75,81.75,...,30.25,7,11.00,7.25,346.50,426.50,222.50,13.25,21.50,16
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Region,ICB Code,ICB Name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Add new headings for date and measure

We can also add the data we stored about the month and measure.

In [ ]:
df['month'] = monthyr
df['measure'] = measure
df.head(5)

,Region,ICB Code,ICB Name,Awaiting a medical decision/intervention including writing the discharge summary,Awaiting community equipment and adaptations to housing,Awaiting confirmation from community hub/single point of access that referral received and actioned,Awaiting Diagnostic test,Awaiting medicines to take home,Awaiting referral to community single point of access,Awaiting therapy decision to discharge,...,Individual/family not in agreement with discharge plans,No Plan,Pathway 1: awaiting availability of resource for assessment and start of care at home,Pathway 2: awaiting availability of rehabilitation bed in community hospital or other bedded setting,Pathway 3: awaiting availability of a bed in a residential or nursing home that is likely to be a permanent placement,Remains in hospital to avoid spread of (non-Covid 19) infectious disease and because there is no other suitable location to discharge to,Repatriation/Transfer to another acute trust for specialist treatment or ongoing treatment,Safeguarding concern preventing discharge or Court of protection,month,measure
0,NaN,NaN,ENGLAND (Type 1 Trusts),411.75,157.00,421.75,135.50,66.75,296.75,439.0,...,94.50,385.75,2040.75,2040.00,1517.25,66.25,140.25,53.5,April 2022,Average number of people per day with length o...
1,NaN,NaN,EAST OF ENGLAND,42.25,30.00,55.00,40.00,25.25,27.25,60.0,...,8.50,50.75,278.50,265.25,108.00,8.50,27.50,2.5,April 2022,Average number of people per day with length o...
2,NaN,NaN,LONDON,61.00,35.75,17.75,3.25,6.75,17.00,49.5,...,22.00,34.00,163.25,143.25,237.75,0.25,33.00,4.75,April 2022,Average number of people per day with length o...
3,NaN,NaN,MIDLANDS,103.25,17.50,56.50,49.00,14.25,47.00,46.5,...,11.50,62.50,295.50,281.25,167.50,9.50,18.75,7,April 2022,Average number of people per day with length o...
4,NaN,NaN,NORTH EAST AND YORKSHIRE,60.50,29.25,79.50,24.75,5.00,41.00,66.0,...,21.25,66.25,374.25,335.75,289.75,14.00,18.75,15,April 2022,Average number of people per day with length o...


## Remove rows

We can now remove that row and the empty one above it.

In [ ]:
#axis = 0 means we want to drop rows
#drop rows 8 and 9 from
df = df.drop(axis = 0, labels = [8,9])
df.head(10)

,Region,ICB Code,ICB Name,Awaiting a medical decision/intervention including writing the discharge summary,Awaiting community equipment and adaptations to housing,Awaiting confirmation from community hub/single point of access that referral received and actioned,Awaiting Diagnostic test,Awaiting medicines to take home,Awaiting referral to community single point of access,Awaiting therapy decision to discharge,...,Individual/family not in agreement with discharge plans,No Plan,Pathway 1: awaiting availability of resource for assessment and start of care at home,Pathway 2: awaiting availability of rehabilitation bed in community hospital or other bedded setting,Pathway 3: awaiting availability of a bed in a residential or nursing home that is likely to be a permanent placement,Remains in hospital to avoid spread of (non-Covid 19) infectious disease and because there is no other suitable location to discharge to,Repatriation/Transfer to another acute trust for specialist treatment or ongoing treatment,Safeguarding concern preventing discharge or Court of protection,month,measure
0,NaN,NaN,ENGLAND (Type 1 Trusts),411.75,157.00,421.75,135.50,66.75,296.75,439.00,...,94.50,385.75,2040.75,2040.00,1517.25,66.25,140.25,53.5,April 2022,Average number of people per day with length o...
1,NaN,NaN,EAST OF ENGLAND,42.25,30.00,55.00,40.00,25.25,27.25,60.00,...,8.50,50.75,278.50,265.25,108.00,8.50,27.50,2.5,April 2022,Average number of people per day with length o...
2,NaN,NaN,LONDON,61.00,35.75,17.75,3.25,6.75,17.00,49.50,...,22.00,34.00,163.25,143.25,237.75,0.25,33.00,4.75,April 2022,Average number of people per day with length o...
3,NaN,NaN,MIDLANDS,103.25,17.50,56.50,49.00,14.25,47.00,46.50,...,11.50,62.50,295.50,281.25,167.50,9.50,18.75,7,April 2022,Average number of people per day with length o...
4,NaN,NaN,NORTH EAST AND YORKSHIRE,60.50,29.25,79.50,24.75,5.00,41.00,66.00,...,21.25,66.25,374.25,335.75,289.75,14.00,18.75,15,April 2022,Average number of people per day with length o...
5,NaN,NaN,NORTH WEST,99.25,17.75,84.00,16.75,11.50,92.00,91.25,...,9.00,103.00,289.25,244.50,256.00,14.50,9.75,5.25,April 2022,Average number of people per day with length o...
6,NaN,NaN,SOUTH EAST,32.25,14.00,27.25,1.50,2.75,18.75,44.00,...,11.25,62.00,293.50,343.50,235.75,6.25,11.00,3,April 2022,Average number of people per day with length o...
7,NaN,NaN,SOUTH WEST,13.25,12.75,101.75,0.25,1.25,53.75,81.75,...,11.00,7.25,346.50,426.50,222.50,13.25,21.50,16,April 2022,Average number of people per day with length o...
10,EAST OF ENGLAND,QUE,NHS CAMBRIDGESHIRE AND PETERBOROUGH INTEGRATED...,11.50,3.50,5.75,36.00,6.00,20.00,13.25,...,3.50,1.75,31.25,31.00,29.75,3.00,9.00,-,April 2022,Average number of people per day with length o...
11,EAST OF ENGLAND,QMM,NHS NORFOLK AND WAVENEY INTEGRATED CARE BOARD,3.25,3.75,9.50,0.50,5.75,4.50,25.25,...,0.50,5.50,87.00,84.50,14.50,3.25,8.00,-,April 2022,Average number of people per day with length o...


## Reshape from wide to long using `melt()`

We have a wide dataframe that we want to narrow, with most column labels moved into a column for 'measure', and the associated values in a column for 'value'.

We can use the `melt()` function for this.

In [ ]:
df.columns

Index(['Region', 'ICB Code', 'ICB Name',
       'Awaiting a medical decision/intervention including writing the discharge summary',
       'Awaiting community equipment and adaptations to housing',
       'Awaiting confirmation from community hub/single point of access that referral received and actioned',
       'Awaiting Diagnostic test', 'Awaiting medicines to take home ',
       'Awaiting referral to community single point of access',
       'Awaiting therapy decision to discharge', 'Awaiting transport',
       'Declared as not meeting the criteria to reside at morning board round and then later in day meets the criteria to reside so discharge stopped ',
       'Homeless/no right of recourse to public funds/no place to discharge to',
       'Individual/family not in agreement with discharge plans', 'No Plan',
       'Pathway 1: awaiting availability of resource for assessment and start of care at home',
       'Pathway 2: awaiting availability of rehabilitation bed in community hos

In [ ]:
#create a list of the indices we want to use
thecols = [0,1,2,21,22]
thecols

[0, 1, 2, 21, 22]

In [ ]:
#keep the columns at the indices we stored - the first 3 and the last two
#use most of the rest of the columns as variables
reshapeddf = pd.melt(df, id_vars=df.columns[thecols],
        value_vars=df.columns[3:21])

reshapeddf

,Region,ICB Code,ICB Name,month,measure,variable,value
0,NaN,NaN,ENGLAND (Type 1 Trusts),April 2022,Average number of people per day with length o...,Awaiting a medical decision/intervention inclu...,411.75
1,NaN,NaN,EAST OF ENGLAND,April 2022,Average number of people per day with length o...,Awaiting a medical decision/intervention inclu...,42.25
2,NaN,NaN,LONDON,April 2022,Average number of people per day with length o...,Awaiting a medical decision/intervention inclu...,61.0
3,NaN,NaN,MIDLANDS,April 2022,Average number of people per day with length o...,Awaiting a medical decision/intervention inclu...,103.25
4,NaN,NaN,NORTH EAST AND YORKSHIRE,April 2022,Average number of people per day with length o...,Awaiting a medical decision/intervention inclu...,60.5
...,...,...,...,...,...,...,...
3109,SOUTH WEST,RK9,UNIVERSITY HOSPITALS PLYMOUTH NHS TRUST,April 2022,Average number of people per day with length o...,Safeguarding concern preventing discharge or C...,-
3110,SOUTH WEST,RN3,GREAT WESTERN HOSPITALS NHS FOUNDATION TRUST,April 2022,Average number of people per day with length o...,Safeguarding concern preventing discharge or C...,-
3111,SOUTH WEST,RNZ,SALISBURY NHS FOUNDATION TRUST,April 2022,Average number of people per day with length o...,Safeguarding concern preventing discharge or C...,-
3112,SOUTH WEST,RTE,GLOUCESTERSHIRE HOSPITALS NHS FOUNDATION TRUST,April 2022,Average number of people per day with length o...,Safeguarding concern preventing discharge or C...,-


## Export as a CSV

In [ ]:
reshapeddf.to_csv("reshapeddf.csv")

## Scrape the other spreadsheet URLs

Now we need to fetch all of the spreadsheets, so we need to scrape the sitreps page. For this scraper code I've used ChatGPT with the prompt:

> *Write a Python script that uses BeautifulSoup to fetch all the links to XLSX files on the webpage https://www.england.nhs.uk/statistics/statistical-work-areas/discharge-delays-acute-data/, and stores them in a list*

I've added my own comments below explaining what the code does.

In [ ]:
#store the URL
url = 'https://www.england.nhs.uk/statistics/statistical-work-areas/discharge-delays-acute-data/'
#fetch the page at that URL
response = requests.get(url)
#convert it into a soup object
soup = BeautifulSoup(response.content, 'html.parser')
#create an empty list
xlsx_links = []
#grab all the <a> tags and loop through them
for link in soup.find_all('a'):
  #grab the href= attribute
    href = link.get('href')
    #if it ends with .xlsx
    if href and href.endswith('.xlsx'):
      #add it to the list
        xlsx_links.append(href)

#check how long that is - it should be 17 months
print(len(xlsx_links))


18


We can print each one to see if they cover all the months we'd expect.

In [ ]:
#loop through the links captured
for i in xlsx_links:
  #print each one
  print(i)

https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2023/08/Daily-discharge-sitrep-monthly-data-webfile-April2022-revised.xlsx
https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2023/08/Daily-discharge-sitrep-monthly-data-webfile-May2022-revised.xlsx
https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2023/08/Daily-discharge-sitrep-monthly-data-webfile-June2022-revised.xlsx
https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2023/08/Daily-discharge-sitrep-monthly-data-webfile-July2022-revised.xlsx
https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2023/08/Daily-discharge-sitrep-monthly-data-webfile-August2022-revised.xlsx
https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2023/08/Daily-discharge-sitrep-monthly-data-webfile-September2022-revised.xlsx
https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2023/08/Daily-discharge-sitrep-monthly-data-webfile-October2022-revised.xlsx
https://www.engla

### Remove the extra file

At the end there's an extra file that covers from April 2021 onwards but it's at the national level with no local breakdown. We need to remove that from the list.

In [ ]:
#replace the list with the same list,
#but only up to (and not including) the last item
xlsx_links = xlsx_links[:-1]

## Store in some functions

Now we know it works we can store in a function and call that on multiple CSV files. First, we need a function to fetch and clean the XLSX.

We need to identify which row has the second headers in, in order to identify which ones to remove.

In [ ]:
#loop through 10 indices
for i in range(0,10):
  #fetch the cell in column 4 at that position
  thiscell = df[df.keys()[4]][i]
  #and print
  print(thiscell)
  #if it is the headin we want
  if thiscell == 'Number of patients who no longer meet the criteria to reside':
    #store the index
    second_header_row = i
    print("The headers are on row ", second_header_row)

Number of patients who no longer meet the criteria to reside
The headers are on row  0
20155
2070
2139
3219
3890
3224
2822
2791
nan


In [ ]:
#let's store the original list of rows to drop
rows_to_drop = [0,1]
#we can now remove from that the second header row if it's one of those
rows_to_drop.remove(second_header_row)
#and check it's worked
rows_to_drop

[1]

### Function 1: removing empty rows and columns (`clean_sitreps`)

Now we can add those lines to a function.

In [ ]:
def clean_sitreps(xlsxurl):
  #read in the sheet called 'Table 5', skipping the first 3 rows
  #see https://stackoverflow.com/questions/35400861/pandas-read-table-usecols-error-with on how to specify a range of cols
  df = pd.read_excel(xlsxurl, skiprows = 3, sheet_name = 'Table 5')
  #loop through 10 indices
  for i in range(0,10):
    #fetch the cell in column 4 at that position
    thiscell = df[df.keys()[4]][i]
    #if it is the headin we want
    if thiscell == 'Number of patients who no longer meet the criteria to reside':
      #store the index
      second_header_row = i
      print("The headers are on row ", second_header_row)
  #store a list of rows to drop, ending before the second header row
  rows_to_drop = list(range(0,second_header_row))
  #and use it to drop that row
  df = df.drop(axis = 0, labels = rows_to_drop)
  #drop column 0
  df = df.drop(axis = 1, labels = df.keys()[0])
  #return to whatever called the function
  return(df)


### Function 2: reshaping (`reshape_sitreps`)

Next we create a function for the reshaping.

In [ ]:
#define a new function that does all the cleaning
def reshape_sitreps(df):
  #create an empty list to store a list of dicts
  rows_list = []
  #loop through each row, starting from row 12 because that's where the STP data begins
  for i in range(11,len(df)):
    #print(i)
    #print(df.iloc[i][2])
    #the first, second and third cell should have the region, STP code and name
    #store each of those
    region = df.iloc[i][0]
    stpcode = df.iloc[i][1]
    stpname = df.iloc[i][2]
    #if it's not an empty cell or header cell
    #print("region", region)
    if region != "Region" and not pd.isna(region):
      #loop through the indices for the other 124 cells in the row
      for cell in range(3,len(df.iloc[0])):
        #start with an empty dict
        record = {}
        #print("cell", cell)
        print("measure", df.iloc[0][cell])
        #store the row 1 heading at that position showing the measure
        measure = df.iloc[0][cell]
        print("numofpatients", df.iloc[i][cell], type(df.iloc[i][cell]))
        #store the value in the cell related to that measure
        numberofpatients = df.iloc[i][cell]
        #print("col", df.keys()[cell])
        #If the column heading - the key - *doesn't* begin with 'Unnamed'
        if str(df.keys()[cell])[:7] != "Unnamed":
          #store it, because it will be the date
          #extract the first 10 chars because the others vary
          thisdate = str(df.keys()[cell])[:10]
          #store the measure in that column too - as it's the total
        if measure == 'Number of patients who no longer meet the criteria to reside':
          nolongermeetcriteria = numberofpatients
        #we add a test that it's not '-', i.e. a string
        #print("IS IT ZERO",nolongermeetcriteria)
        if type(nolongermeetcriteria) != str:
          #we check that the total patients is not zero, which would cause an error
          if nolongermeetcriteria != 0:
            #print("NOT ZERO")
            #calculate the patients as a percentage of the total no longer meeting the criteria
            percof = int(numberofpatients)/int(nolongermeetcriteria)
        #if if is zero or '-' then we store the lack of data
        else:
          print("ZERO")
          percof = 'no data'
        #print("thisdate",thisdate)
        #store all of that in the dict
        record['region'] = region
        record['code'] = stpcode
        record['org'] = stpname
        record['date'] = thisdate
        record['measure'] = measure
        record['numberofpatients'] = numberofpatients
        #extra column showing figures as percentages
        record['percof_nolongermeetcriteria'] = percof
        print(record)
        #add to the list
        rows_list.append(record)
        #print("rows_list",rows_list)
  #create a dataframe from that list of dicts
  reshapeddf = pd.DataFrame(rows_list)
  #return the dataframe and list of dicts
  return(reshapeddf, rows_list)

## Run the functions on our scraped URLs

Now let's try to run those on the list of URLs.

In [ ]:
#create a list
listofdfs = []

#loop through the URLs
for i in xlsx_links:
  #run the cleaning function that grabs the XLSX and cleans it
  cleandf = clean_sitreps(i)
  #use the results with the reshaping function
  df1, list1 = reshape_sitreps(cleandf)
  listofdfs.append(df1)

concatenateddf = pd.concat(listofdfs)

Streaming output truncated to the last 5000 lines.
numofpatients 86 <class 'int'>
{'region': 'SOUTH EAST', 'code': 'RVV', 'org': 'EAST KENT HOSPITALS UNIVERSITY NHS FOUNDATION TRUST', 'date': '2023-07-03', 'measure': 'Number of patients discharged', 'numberofpatients': 86, 'percof_nolongermeetcriteria': 0.26299694189602446}
measure Number of patients remaining in hospital who no longer meet the criteria to reside
numofpatients 241 <class 'int'>
{'region': 'SOUTH EAST', 'code': 'RVV', 'org': 'EAST KENT HOSPITALS UNIVERSITY NHS FOUNDATION TRUST', 'date': '2023-07-03', 'measure': 'Number of patients remaining in hospital who no longer meet the criteria to reside', 'numberofpatients': 241, 'percof_nolongermeetcriteria': 0.7370030581039755}
measure Number of patients who no longer meet the criteria to reside
numofpatients 348 <class 'int'>
{'region': 'SOUTH EAST', 'code': 'RVV', 'org': 'EAST KENT HOSPITALS UNIVERSITY NHS FOUNDATION TRUST', 'date': '2023-07-04', 'measure': 'Number of patient

### Adapt the `reshape_sitreps` function to deal with notes (no longer needed)

In the June data in the release before the revisions the function broke with this error:

`ValueError: cannot convert float NaN to integer`

The line of code causing that is this:

`percof = int(numberofpatients)/int(nolongermeetcriteria)`

We can also see in the printed lines, this:

`numofpatients nan <class 'float'>`

...where before it was a number.

And in a previous line:

`region Note: No data are available for 19th June 2022 due to a data collection error.`

This isn't what we're expecting (a region), and checking that spreadsheet shows that this line has a note and no data in the other cells, which is where the `NaN` would come from (and when it tries to convert that to an integer, it generates the error).

We can add another condition to the initial `if` statement which only loops through the cells if the STP name column has a string in it:

`if region != "Region" and not pd.isna(region) and isinstance(stpname, str):`

To capture the note we added an `elif` near the end of the code which runs if the STP Name column is empty but the region column is not:

`elif isinstance(stpname, str) is False and isinstance(region, str) is True:`

And we added a final `else` for empty rows.

*Note: this code is now not run*

In [ ]:
#define a new function that does all the cleaning
def reshape_sitreps(df):
  #create an empty list to store a list of dicts
  rows_list = []
  #loop through each row, starting from row 12 because that's where the STP data begins
  for i in range(11,len(df)):
    #print(i)
    #print(df.iloc[i][2])
    #the first, second and third cell should have the region, STP code and name
    #store each of those
    region = df.iloc[i][0]
    stpcode = df.iloc[i][1]
    stpname = df.iloc[i][2]
    #print("region", region)
    #if it's not an empty cell or header cell or notes row OR the STP Name is empty
    if region != "Region" and not pd.isna(region) and isinstance(stpname, str):
      #loop through the indices for the other 124 cells in the row
      for cell in range(3,len(df.iloc[0])):
        #start with an empty dict
        record = {}
        #print("cell", cell)
        #print("measure", df.iloc[0][cell])
        #store the row 1 heading at that position showing the measure
        measure = df.iloc[0][cell]
        #print("numofpatients", df.iloc[i][cell], type(df.iloc[i][cell]))
        #store the value in the cell related to that measure
        numberofpatients = df.iloc[i][cell]
        #print("col", df.keys()[cell])
        #If the column heading - the key - *doesn't* begin with 'Unnamed'
        if str(df.keys()[cell])[:7] != "Unnamed":
          #store it, because it will be the date
          #extract the first 10 chars because the others vary
          thisdate = str(df.keys()[cell])[:10]
          #store the measure in that column too - as it's the total
        if measure == 'Number of patients who no longer meet the criteria to reside':
          nolongermeetcriteria = numberofpatients
        #we check that the total patients is not zero, which would cause an error
        #print("IS IT ZERO",nolongermeetcriteria)
        if nolongermeetcriteria != 0:
          #print("NOT ZERO")
          #calculate the patients as a percentage of the total no longer meeting the criteria
          percof = int(numberofpatients)/int(nolongermeetcriteria)
        #if if is zero then we store the lack of data
        else:
          #print("ZERO")
          percof = 'no data'
        #print("thisdate",thisdate)
        #store all of that in the dict
        record['region'] = region
        record['code'] = stpcode
        record['org'] = stpname
        record['date'] = thisdate
        record['measure'] = measure
        record['numberofpatients'] = numberofpatients
        #extra column showing figures as percentages
        record['percof_nolongermeetcriteria'] = percof
        #create a note field so it's consistent
        record['note'] = ''
        #print(record)
        #add to the list
        rows_list.append(record)
        #print("rows_list",rows_list)
    #if STP Name is not a string (it's empty because it's a notes row)
    #but the region is a string because it is not empty
    elif isinstance(stpname, str) is False and isinstance(region, str) is True:
      record['region'] = ''
      record['code'] = ''
      record['org'] = ''
      record['date'] = thisdate
      record['measure'] = 'note'
      record['numberofpatients'] = ''
      record['note'] = region
      #extra column showing figures as percentages
      record['percof_nolongermeetcriteria'] = ''
      #print(record)
      #add to the list
      rows_list.append(record)
    #else:
      #print("EMPTY ROW/HEADER ROW")

  #create a dataframe from that list of dicts
  reshapeddf = pd.DataFrame(rows_list)
  #return the dataframe and list of dicts
  return(reshapeddf, rows_list)

In [ ]:
xlsx_links[2]
cleandf = clean_sitreps(xlsx_links[2])
df1, list1 = reshape_sitreps(cleandf)

Streaming output truncated to the last 5000 lines.
ZERO
{'region': 'SOUTH WEST', 'code': 'RH5', 'org': 'SOMERSET NHS FOUNDATION TRUST', 'date': '2022-06-19', 'measure': 'Number of patients discharged by 17:00', 'numberofpatients': 0, 'percof_nolongermeetcriteria': 'no data', 'note': ''}
measure Number of patients discharged between 17:01 and 23:59
numofpatients 0 <class 'int'>
ZERO
{'region': 'SOUTH WEST', 'code': 'RH5', 'org': 'SOMERSET NHS FOUNDATION TRUST', 'date': '2022-06-19', 'measure': 'Number of patients discharged between 17:01 and 23:59', 'numberofpatients': 0, 'percof_nolongermeetcriteria': 'no data', 'note': ''}
measure Number of patients remaining in hospital who no longer meet the criteria to reside
numofpatients 0 <class 'int'>
ZERO
{'region': 'SOUTH WEST', 'code': 'RH5', 'org': 'SOMERSET NHS FOUNDATION TRUST', 'date': '2022-06-19', 'measure': 'Number of patients remaining in hospital who no longer meet the criteria to reside', 'numberofpatients': 0, 'percof_nolongermeet

## Run the revised function on all URLs (no longer needed)

Now we try that on all the files again.

*Note: this is no longer run*.

In [ ]:
#create a list
listofdfs = []

#loop through the URLs
for i in xlsx_links:
  print('scraping', i)
  #run the cleaning function that grabs the XLSX and cleans it
  cleandf = clean_sitreps(i)
  #use the results with the reshaping function
  df1, list1 = reshape_sitreps(cleandf)
  listofdfs.append(df1)

concatenateddf = pd.concat(listofdfs)

## Export the combined dataframe as a CSV

Once done, we can export to the Files area.

In [ ]:
concatenateddf.to_csv("concatenateddf.csv")

In [ ]:
#import a library for file downloads in Colab
from google.colab import files
#start a file download of that CSV
files.download("concatenateddf.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Quality checking

We can quality check in Excel, but let's do some basic checking here.

In [ ]:
#remind ourselves what the column headings (keys) are
concatenateddf.keys()

Index(['region', 'code', 'org', 'date', 'measure', 'numberofpatients',
       'percof_nolongermeetcriteria'],
      dtype='object')

There should only be three measures, for example. And there should be the same number of values for each.

*Note: previously there were were
four measures, with discharges split between 'Those discharged between 1700 and 2359' and 'Those discharged before 1700'. This level of detail is not available in the revised data.*

In [ ]:
#how many of each measure
concatenateddf['measure'].value_counts()

Number of patients who no longer meet the criteria to reside                          79228
Number of patients discharged                                                         79228
Number of patients remaining in hospital who no longer meet the criteria to reside    79228
Name: measure, dtype: int64

### No longer needed: Cleaning auto-incremented numbers

In the previous April spreadsheet we could see that someone has copied values across columns but not realised that Excel's auto-formatting has incremented the numbers each time. We needed to clean that.

We used regex and the `.sub()` function to do this.

*Note: no longer used*.

In [ ]:
#store an example of a string we need to fix
teststr = 'Number of patients discharged between 17:01 and 23:70'
#test the regex to replace the mistake - 23: followed by two digits - with 23:59
re.sub('23:[0-9]{2}', '23:59', teststr)

'Number of patients discharged between 17:01 and 23:59'

In [ ]:
#store an example of the other string we need to fix
teststr = 'Number of patients discharged by 17:00'
#test the regex to replace the mistake - 17: followed by two digits - with 17:00
#we need the text before it too, otherwise we'd end up replacing 17:01 in the other string
re.sub('discharged by 17:[0-9]{2}', 'discharged by 17:00', teststr)

'Number of patients discharged by 17:00'

Once we've got the right regex, we clean the whole column.

In [ ]:
#apply that to all the values in the column
cleanmeasures1 = [re.sub('discharged by 17:[0-9]{2}', 'discharged by 17:00', i) for i in concatenateddf['measure']]
#check the results - we have to convert to a pandas Series to apply value_counts()
pd.Series(cleanmeasures1).value_counts()

Number of patients who no longer meet the criteria to reside                          69407
Number of patients discharged by 17:00                                                69407
Number of patients remaining in hospital who no longer meet the criteria to reside    69406
Number of patients discharged between 17:01 and 23:59                                 64843
Number of patients discharged between 17:01 and 23:81                                   163
Number of patients discharged between 17:01 and 23:76                                   163
Number of patients discharged between 17:01 and 23:77                                   163
Number of patients discharged between 17:01 and 23:78                                   163
Number of patients discharged between 17:01 and 23:79                                   163
Number of patients discharged between 17:01 and 23:80                                   163
Number of patients discharged between 17:01 and 23:82                           

That's fixed one. Now for the other.

In [ ]:
#apply that to all the values in the half-cleaned list
cleanmeasures2 = [re.sub('23:[0-9]{2}', '23:59', i) for i in cleanmeasures1]
#check the results - we have to convert to a pandas Series to apply value_counts()
pd.Series(cleanmeasures2).value_counts()

Number of patients who no longer meet the criteria to reside                          69407
Number of patients discharged by 17:00                                                69407
Number of patients discharged between 17:01 and 23:59                                 69407
Number of patients remaining in hospital who no longer meet the criteria to reside    69406
note                                                                                      2
dtype: int64

Now to add it back to the dataframe. We could overwrite the original column but we are going to add it as a 'clean version' so the two can be compared and checked.

In [ ]:
#create a new column in the dataframe, and assign the list of cleaned measures
concatenateddf['measure_cleaned'] = cleanmeasures2
#check
concatenateddf['measure_cleaned'].value_counts()

Number of patients who no longer meet the criteria to reside                          69407
Number of patients discharged by 17:00                                                69407
Number of patients discharged between 17:01 and 23:59                                 69407
Number of patients remaining in hospital who no longer meet the criteria to reside    69406
note                                                                                      2
Name: measure_cleaned, dtype: int64

## Export cleaned version

We repeat the code from earlier.

In [ ]:
concatenateddf.to_csv("concatenateddf.csv")
#import a library for file downloads in Colab
from google.colab import files
#start a file download of that CSV
files.download("concatenateddf.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Add week numbers and days

We can add week numbers and weekdays in a spreadsheet by using the `WEEKNUM` and `TEXT` functions, but let's try to do it here.

[This Stackoverflow thread](https://stackoverflow.com/questions/31181295/converting-a-pandas-date-to-week-number) explains how to get the week number using `.dt.isocalendar().week` and [this one](https://stackoverflow.com/questions/30222533/create-a-day-of-week-column-in-a-pandas-dataframe-using-python) points to `.dt.day_name()` for grabbing the weekday.

However, we need to check if our dates are in datetime format - and they're not.

In [ ]:
#loop through the first 10 items in the date column
for i in concatenateddf.date[:10]:
  #print the item, and the type
  print(i, type(i))

2022-04-01 <class 'str'>
2022-04-01 <class 'str'>
2022-04-01 <class 'str'>
2022-04-02 <class 'str'>
2022-04-02 <class 'str'>
2022-04-02 <class 'str'>
2022-04-03 <class 'str'>
2022-04-03 <class 'str'>
2022-04-03 <class 'str'>
2022-04-04 <class 'str'>


We can convert those strings using the `.to_datetime()` function in pandas.

In [ ]:
#loop through the first 3
for i in concatenateddf.date[:3]:
  #apply .to_datetime() and print
  print(pd.to_datetime(i))

2022-04-01 00:00:00
2022-04-01 00:00:00
2022-04-01 00:00:00


In [ ]:
#create a list by doing this for all items
datetimecolumn = [pd.to_datetime(i) for i in concatenateddf.date]
#check the length of that list
len(datetimecolumn) == len(concatenateddf)

True

In [ ]:
#check it's the same length as the dataframe
len(datetimecolumn) == len(concatenateddf)

True

In [ ]:
#add to the dataframe
concatenateddf['datetime'] = datetimecolumn

In [ ]:
#add a weeknum column
concatenateddf['weeknum'] = concatenateddf['datetime'].dt.isocalendar().week
#add a weekday column
concatenateddf['weekday'] = concatenateddf['datetime'].dt.day_name()
#add a year column
concatenateddf['year'] = concatenateddf['datetime'].dt.year


### Add a year-weeknum column

Because some week numbers appear twice, we want to create a column where they are combined with years to identify which year they belong to (and sort appropriately).

To do this we need to convert both to strings, and also prefix the week numbers with a zero where they are one digit (so that they will sort properly).

In [ ]:
#create an empty list
weeknumstr = []
#convert weeknum into a string with a 0 prefix if needed for sorting
for i in concatenateddf['weeknum']:
  if(i < 10):
    i = '0'+str(i)
  else:
    i = str(i)
  weeknumstr.append(i)
#check the length
len(weeknumstr)

237684

In [ ]:
#create a list of year strings
yrstr = [str(i) for i in concatenateddf['year']]
#check the length
len(yrstr)

237684

In [ ]:
#create an empty list
yr_weeknumlist = []
#loop through a list of indices
for i in range(0,len(yrstr)):
  #add to that list the year-weeknum string at each index
  yr_weeknumlist.append(yrstr[i]+weeknumstr[i])

len(yr_weeknumlist)

237684

In [ ]:
#add to the dataframe
concatenateddf['yr_weeknum'] = yr_weeknumlist

In [ ]:
concatenateddf

,region,code,org,date,measure,numberofpatients,percof_nolongermeetcriteria,datetime,weeknum,weekday,year,yr_weeknum
0,EAST OF ENGLAND,QUE,NHS CAMBRIDGESHIRE AND PETERBOROUGH INTEGRATED...,2022-04-01,Number of patients who no longer meet the crit...,426,1.0,2022-04-01,13,Friday,2022,202213
1,EAST OF ENGLAND,QUE,NHS CAMBRIDGESHIRE AND PETERBOROUGH INTEGRATED...,2022-04-01,Number of patients discharged,230,0.539906,2022-04-01,13,Friday,2022,202213
2,EAST OF ENGLAND,QUE,NHS CAMBRIDGESHIRE AND PETERBOROUGH INTEGRATED...,2022-04-01,Number of patients remaining in hospital who n...,196,0.460094,2022-04-01,13,Friday,2022,202213
3,EAST OF ENGLAND,QUE,NHS CAMBRIDGESHIRE AND PETERBOROUGH INTEGRATED...,2022-04-02,Number of patients who no longer meet the crit...,440,1.0,2022-04-02,13,Saturday,2022,202213
4,EAST OF ENGLAND,QUE,NHS CAMBRIDGESHIRE AND PETERBOROUGH INTEGRATED...,2022-04-02,Number of patients discharged,208,0.472727,2022-04-02,13,Saturday,2022,202213
...,...,...,...,...,...,...,...,...,...,...,...,...
14968,SOUTH WEST,RVJ,NORTH BRISTOL NHS TRUST,2023-07-30,Number of patients discharged,46,0.172285,2023-07-30,30,Sunday,2023,202330
14969,SOUTH WEST,RVJ,NORTH BRISTOL NHS TRUST,2023-07-30,Number of patients remaining in hospital who n...,221,0.827715,2023-07-30,30,Sunday,2023,202330
14970,SOUTH WEST,RVJ,NORTH BRISTOL NHS TRUST,2023-07-31,Number of patients who no longer meet the crit...,333,1.0,2023-07-31,31,Monday,2023,202331
14971,SOUTH WEST,RVJ,NORTH BRISTOL NHS TRUST,2023-07-31,Number of patients discharged,103,0.309309,2023-07-31,31,Monday,2023,202331


### ***NOTE: Week 52 is split between years***

When quality checking this data in a spreadsheet, one quirk becomes apparent: Sunday January 1 2023 is week 52 - officially the last week of 2022. But of course the year is not 2022. As a result it needs cleaning.

In [ ]:
#replace any instances of 202352 with 202252
concatenateddf['yr_weeknum'] = [i.replace('202352','202252') for i in concatenateddf['yr_weeknum']]

## Export version with week numbers and days

In [ ]:
concatenateddf.to_csv("concatenateddf.csv")
#import a library for file downloads in Colab
from google.colab import files
#start a file download of that CSV
files.download("concatenateddf.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Filter out the STP

This has made the file a lot larger. We can make it smaller by filtering out the STPs (we only want to look at the trusts).

In [ ]:
#we have published a lookup list of STPs and trusts
lookupurl = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTAJFNix2fyEcqrAj08mYzvG8tHr6xVsMklKRnXRdO6cPDZHuPcSePbSZDaq_MFvXo3K3TSBNZLyiXM/pub?gid=389189088&single=true&output=csv"
lookupdf = pd.read_csv(lookupurl)
lookupdf

,Region,STP Code,STP/org Name,Type
0,EAST OF ENGLAND,QUE,CAMBRIDGESHIRE AND PETERBOROUGH,STP
1,EAST OF ENGLAND,QMM,NORFOLK AND WAVENEY HEALTH AND CARE PARTNERSHIP,STP
2,EAST OF ENGLAND,QJG,SUFFOLK AND NORTH EAST ESSEX,STP
3,EAST OF ENGLAND,QHG,"BEDFORDSHIRE, LUTON AND MILTON KEYNES",STP
4,EAST OF ENGLAND,QM7,HERTFORDSHIRE AND WEST ESSEX,STP
...,...,...,...,...
158,SOUTH WEST,RK9,UNIVERSITY HOSPITALS PLYMOUTH NHS TRUST,TRUST
159,SOUTH WEST,RN3,GREAT WESTERN HOSPITALS NHS FOUNDATION TRUST,TRUST
160,SOUTH WEST,RNZ,SALISBURY NHS FOUNDATION TRUST,TRUST
161,SOUTH WEST,RTE,GLOUCESTERSHIRE HOSPITALS NHS FOUNDATION TRUST,TRUST


In [ ]:
concatenateddf.keys()

Index(['region', 'code', 'org', 'date', 'measure', 'numberofpatients',
       'percof_nolongermeetcriteria', 'datetime', 'weeknum', 'weekday', 'year',
       'yr_weeknum'],
      dtype='object')

In [ ]:
#merge the dataframes on the code column
mergeddf = concatenateddf.merge(lookupdf, left_on = 'code', right_on = 'STP Code', how = 'left')
mergeddf

,region,code,org,date,measure,numberofpatients,percof_nolongermeetcriteria,datetime,weeknum,weekday,year,yr_weeknum,Region,STP Code,STP/org Name,Type
0,EAST OF ENGLAND,QUE,NHS CAMBRIDGESHIRE AND PETERBOROUGH INTEGRATED...,2022-04-01,Number of patients who no longer meet the crit...,426,1.0,2022-04-01,13,Friday,2022,202213,EAST OF ENGLAND,QUE,CAMBRIDGESHIRE AND PETERBOROUGH,STP
1,EAST OF ENGLAND,QUE,NHS CAMBRIDGESHIRE AND PETERBOROUGH INTEGRATED...,2022-04-01,Number of patients discharged,230,0.539906,2022-04-01,13,Friday,2022,202213,EAST OF ENGLAND,QUE,CAMBRIDGESHIRE AND PETERBOROUGH,STP
2,EAST OF ENGLAND,QUE,NHS CAMBRIDGESHIRE AND PETERBOROUGH INTEGRATED...,2022-04-01,Number of patients remaining in hospital who n...,196,0.460094,2022-04-01,13,Friday,2022,202213,EAST OF ENGLAND,QUE,CAMBRIDGESHIRE AND PETERBOROUGH,STP
3,EAST OF ENGLAND,QUE,NHS CAMBRIDGESHIRE AND PETERBOROUGH INTEGRATED...,2022-04-02,Number of patients who no longer meet the crit...,440,1.0,2022-04-02,13,Saturday,2022,202213,EAST OF ENGLAND,QUE,CAMBRIDGESHIRE AND PETERBOROUGH,STP
4,EAST OF ENGLAND,QUE,NHS CAMBRIDGESHIRE AND PETERBOROUGH INTEGRATED...,2022-04-02,Number of patients discharged,208,0.472727,2022-04-02,13,Saturday,2022,202213,EAST OF ENGLAND,QUE,CAMBRIDGESHIRE AND PETERBOROUGH,STP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237679,SOUTH WEST,RVJ,NORTH BRISTOL NHS TRUST,2023-07-30,Number of patients discharged,46,0.172285,2023-07-30,30,Sunday,2023,202330,SOUTH WEST,RVJ,NORTH BRISTOL NHS TRUST,TRUST
237680,SOUTH WEST,RVJ,NORTH BRISTOL NHS TRUST,2023-07-30,Number of patients remaining in hospital who n...,221,0.827715,2023-07-30,30,Sunday,2023,202330,SOUTH WEST,RVJ,NORTH BRISTOL NHS TRUST,TRUST
237681,SOUTH WEST,RVJ,NORTH BRISTOL NHS TRUST,2023-07-31,Number of patients who no longer meet the crit...,333,1.0,2023-07-31,31,Monday,2023,202331,SOUTH WEST,RVJ,NORTH BRISTOL NHS TRUST,TRUST
237682,SOUTH WEST,RVJ,NORTH BRISTOL NHS TRUST,2023-07-31,Number of patients discharged,103,0.309309,2023-07-31,31,Monday,2023,202331,SOUTH WEST,RVJ,NORTH BRISTOL NHS TRUST,TRUST


### Drop the STPs

Now we can filter out those where the type is 'trust'.

In [ ]:
#filter where the type is trust
trustsonly = mergeddf[mergeddf['Type'] == 'TRUST']
#check the resulting length
len(trustsonly)


176322

### Drop columns

And drop the columns we no longer need.

In [ ]:
trustsonly.keys()

Index(['region', 'code', 'org', 'date', 'measure', 'numberofpatients',
       'percof_nolongermeetcriteria', 'datetime', 'weeknum', 'weekday', 'year',
       'yr_weeknum', 'Region', 'STP Code', 'STP/org Name', 'Type'],
      dtype='object')

In [ ]:
#drop the columns we don't need
trustsonly = trustsonly.drop(columns = ['Region', 'STP Code', 'STP/org Name', 'Type'])

## Export trusts only version

In [ ]:
trustsonly.to_csv("trustsonly.csv")
#import a library for file downloads in Colab
from google.colab import files
#start a file download of that CSV
files.download("trustsonly.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>